In [ ]:
from tkinter import *
import requests
import json
from bs4 import BeautifulSoup
import sqlite3



def cerrarConexion(conexion):
    conexion.commit()
    conexion.close()

def abrirConexion():
    return sqlite3.connect('Guguel.db')

def crearTabla():
    conexion = abrirConexion()
    cursor = conexion.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS INFORMACION (
            ID INTEGER PRIMARY KEY,
            BUSQUEDA VARCHAR(50),
            NOMBRE VARCHAR(80),
            ENLACE VARCHAR(255),
            CONTENIDO TEXT
        )
    """)
    cerrarConexion(conexion)

def comprobarContenido(nombre,busqueda):
    conexion = abrirConexion()
    cursor = conexion.cursor()
    cursor.execute("SELECT NOMBRE FROM INFORMACION WHERE BUSQUEDA = ?", (busqueda,))
    resultados = cursor.fetchall()
    
    for resultado in resultados:
        if ("".join(resultado)).upper()==nombre.upper():
            cerrarConexion(conexion)
            return True
            
    cerrarConexion(conexion)
    return False

def mostrarContenido(nombre,busqueda):
    conexion = abrirConexion()
    cursor = conexion.cursor()
    contenidoBusqueda.delete("1.0", "end")

    cursor.execute("SELECT NOMBRE, CONTENIDO FROM INFORMACION WHERE BUSQUEDA = ?", (busqueda.upper(),))
    contenido = cursor.fetchall()

    contenidoBusqueda.tag_config("nombre", background="white", font=("Helvetica", 15, "bold"))
    
    for tema in contenido:
        nombre, contenido_texto = tema 
        contenidoBusqueda.insert("end", f"     {nombre.upper()} \n", "nombre")
        contenidoBusqueda.insert("end", f"{contenido_texto}\n")
        
    cerrarConexion(conexion)
        

def almacenarContenido(nombre,direccion,contenido,busqueda):
    conexion = abrirConexion()
    cursor = conexion.cursor()
    
    cursor.execute("INSERT INTO INFORMACION (BUSQUEDA, NOMBRE, ENLACE, CONTENIDO) VALUES (?, ?, ?, ?)", 
    (busqueda, nombre, direccion, contenido))
    
    cerrarConexion(conexion)

crearTabla()

def extraerTematicaApiWiki(busqueda):
    if busqueda.strip() == "":
        contenidoBusqueda.delete("1.0", "end")
        contenidoBusqueda.insert("end", "ERROR: No se puede buscar con el campo de busqueda en blanco\n")
        return 1

    conexion = abrirConexion()
    cursor = conexion.cursor()
    cursor.execute("SELECT COUNT(*) FROM INFORMACION WHERE BUSQUEDA = ?", (busqueda.upper(),))
    nRes = cursor.fetchone()[0]
    cerrarConexion(conexion)

    if nRes > 0:
        mostrarContenido("", busqueda)
    else:
        encontrados = {}
        sesion = requests.Session()
        direccion = "https://es.wikipedia.org/w/api.php"
        parametros = {
            "action": "opensearch",
            "namespace": "0",
            "search": busqueda,
            "limit": "10",
            "format": "json"
        }
        resultado = sesion.get(url=direccion, params=parametros)
        datos = resultado.json()
        nombres = datos[1]
        direcciones = datos[3]

        for nombre, direccion in zip(nombres, direcciones):
            extraerBusqueda(nombre, direccion)

def extraerBusqueda(nombre,direccion):
    peticion = requests.get(direccion)
    sopa = BeautifulSoup(peticion.text, "html.parser")
    contenido = sopa.find_all("p")
    parrafos=[parrafo.get_text() for parrafo in contenido]
    resultados={nombre:parrafos}
    busqueda=campoBusqueda.get("1.0", "end-1c")
    if not comprobarContenido(nombre,busqueda):
        for nombreRes, parrafos in resultados.items():
            almacenarContenido(nombreRes,direccion,"\n".join(parrafos),busqueda.upper())
    mostrarContenido(nombre,busqueda)

def buscarEnContenido():
    listaResultados = {}
    contenidoBusqueda.tag_remove("resultado-encontrado", "1.0", "end")
    encontrado = False

    busqueda = campoRegex.get("1.0", "end-1c").strip()
    contenido = contenidoBusqueda.get("1.0", "end-1c")

    if busqueda:
        posicionEnTexto = "1.0"
        while True:
            posicionEnTexto = contenidoBusqueda.search(busqueda, posicionEnTexto, stopindex="end", regexp=True)
            if not posicionEnTexto:
                break

            if not encontrado:
                primerResultado = posicionEnTexto
                encontrado = True

            inicioParrafo = contenidoBusqueda.index(f"{posicionEnTexto} linestart")
            finParrafo = contenidoBusqueda.index(f"{posicionEnTexto} lineend")

            contenidoBusqueda.tag_add("resultado-encontrado", inicioParrafo, finParrafo)
            listaResultados.update({inicioParrafo: finParrafo})

            posicionEnTexto = finParrafo

        if encontrado:
            contenidoBusqueda.see(primerResultado)
            contenidoBusqueda.tag_config("resultado-encontrado", background="yellow")

    return listaResultados
        
def convertirAHTML():
    temas=buscarEnContenido()
    busqueda=campoRegex.get("1.0", "end-1c").strip()
    
    documento=f'''
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>Resultados de {busqueda.capitalize()}</title>
            </head>
            <body style="font-family:arial; background-color:gray; background-position: center;
            background-image:url('https://wallpapercat.com/w/full/2/7/7/41442-3840x2160-desktop-4k-google-background.jpg');">
            <h1 style="text-align:center; color:rgb(40,40,40);">
            <u style="color:rgb(60,60,60);"> <span style="color:rgb(75,75,200);">Resultados de </span><span style="color:rgb(200,75,75);">{busqueda.capitalize()}</span>
            <span style="color:rgb(200,200,20);">encontrados en la </span><span style="color:rgb(50,200,50);">temática de {campoBusqueda.get("1.0", "end-1c").strip()}</span></u></h1>
            '''
    for puntoInicio,puntoFin in temas.items():
        documento+=f'''
                <p style="
                color:rgb(170,170,170); background-color:rgb(23,23,23,0.95); border-radius:2w;
                padding:1vw; margin:0.4w; text-align:justify; font-size:1.4vw;
                border-radius:1.5vw;">{contenidoBusqueda.get(puntoInicio,puntoFin).strip().capitalize()}
                </p>
                '''
        
    documento+= """     
            </body>
            </html>
            """
    with open("resultados.html", "w", encoding="utf-8") as html:
        html.write(documento)
    
        
ventanaInicial = Tk()
ventanaInicial.title("Guguel Cromo")
ventanaInicial.resizable(0, 0)
ventanaInicial.geometry("700x550+400+100")
ventanaInicial.configure(bg="#4F4F4F")
ventanaInicial.iconbitmap("Guguelcron.ico")

imagen = PhotoImage(file="guguel.png").subsample(4)
Label(ventanaInicial, image=imagen, bg="#4F4F4F").grid(row=1, column=3, pady=40)

campoBusqueda = Text(ventanaInicial, bg="#4F4F4F", fg="#CFCFCF", width=20, height=1, font=("Helvetica", 18))
campoBusqueda.grid(row=2, column=3)

iconoBuscar = PhotoImage(file="buscar.png").subsample(12)
botonBusqueda = Button(ventanaInicial, text="Busqueda", bg="#7F7F7F", image=iconoBuscar, 
                       command=lambda: extraerTematicaApiWiki(campoBusqueda.get("1.0", "end-1c")))
botonBusqueda.grid(row=2, column=4, padx=4, sticky="ws")

Label(ventanaInicial, text="Resultados", bg="#4F4F4F", fg="#CFCFCF", font=("Helvetica", 18)).grid(row=3, column=3, pady=15)

frame = Frame(ventanaInicial)
frame.grid(row=4, column=1, columnspan=10, pady=4, padx=85, sticky="nsew")

contenidoBusqueda = Text(frame, width=45, height=7, bg="#7F7F7F", fg="#000000",font=("Helvetica", 15))
contenidoBusqueda.grid(row=0, column=0, sticky="nsew")

scrollFrame = Scrollbar(frame, orient="vertical", bg="#4F4F4F", command=contenidoBusqueda.yview)
scrollFrame.grid(row=0, column=1, sticky="ns")
contenidoBusqueda.config(yscrollcommand=scrollFrame.set)

Label(ventanaInicial, text="Busqueda", bg="#4F4F4F", fg="#CFCFCF", font=("Helvetica", 16)).grid(row=5, column=2, sticky="e")

campoRegex = Text(ventanaInicial, bg="#4F4F4F", fg="#CFCFCF", width=20, height=1, font=("Helvetica", 18))
campoRegex.grid(row=5, column=3,pady="25")

iconoDestacar = PhotoImage(file="destacar.png").subsample(7)
botonBusqueda = Button(ventanaInicial, text="Destacar", bg="#7F7F7F", image=iconoDestacar,
                       command=buscarEnContenido)
botonBusqueda.grid(row=5, column=4, sticky="w")



iconoHTML = PhotoImage(file="HTML5.png").subsample(42)
botonConversion = Button(ventanaInicial, text="Conversion HTML", bg="#7F7F7F", command=convertirAHTML, image=iconoHTML)
botonConversion.grid(row=5, padx=50,column=4)

Label(ventanaInicial, bg="#4F4F4F").grid(row=0, column=1, pady=0, sticky="n", columnspan=10)
Label(ventanaInicial, bg="#4F4F4F").grid(row=1, column=0, padx=240, sticky="es", columnspan=4)
ventanaInicial.mainloop()
